* LSGAN使ってグラタンに似た偽画像を生成する

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# カレントディレクトリの読み込みとカレントディレクトリへの移動
import sys
sys.path.append(f'/content/drive/My Drive/system/')
import os
os.chdir(f'/content/drive/My Drive/system/myanswer')

In [3]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-npuh445k
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-npuh445k
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101066 sha256=8684da379da894bbbee55b27f492276a8b2f6bb0d506d3c88574d65ef1a7fcd4
  Stored in directory: /tmp/pip-ephem-wheel-cache-7p9ul7ut/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [4]:
!pip install scipy==1.1.0

In [5]:
from __future__ import print_function, division
from keras.datasets import mnist
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from glob import glob
from skimage.transform import resize 
import datetime
import pickle
import matplotlib.pyplot as plt
import numpy as np
import scipy
import scipy.misc
import imageio

In [6]:
class DataLoader():
    def __init__(self, dataset_name, img_res=(128, 128)):
        self.dataset_name = dataset_name
        self.img_res = img_res

    def load_data(self, is_testing=False):
        if os.path.exists("../pickle/{}_tensor.pickle".format(self.dataset_name)):
            with open("../pickle/{}_tensor.pickle".format(self.dataset_name), 'rb') as p:
                imgs = pickle.load(p)
        else:
            img_pathes = glob('../figure/foodimg128/%s/*.jpg' % (self.dataset_name))
            imgs = []
            for img_path in img_pathes:
                img = self.imread(img_path)
                if not is_testing:
                    img = scipy.misc.imresize(img, self.img_res)
                    if np.random.random() > 0.5:
                        img = np.fliplr(img)
                else:
                    img = scipy.misc.imresize(img, self.img_res)
                imgs.append(img)
            with open('../pickle/{}_tensor.pickle'.format(self.dataset_name), 'wb') as p:
                pickle.dump(imgs , p)

        return np.array(imgs)

    def imread(self, path):
        return scipy.misc.imread(path, mode="RGB").astype(np.float)

In [7]:
class LSGAN():
    def __init__(self, dataset_name="mnist"):
        self.img_rows = 28
        self.img_cols = 28 
        self.dataset_name = dataset_name
        # 変換させたい画像のデータセットの名前を指定
        if self.dataset_name == "mnist":
            self.channels = 1
        else:
            self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        self.data_loader = DataLoader(dataset_name=self.dataset_name,
                                      img_res=(self.img_rows, self.img_cols))

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='mse',
            optimizer=optimizer,
            metrics=['accuracy'])
        

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generated imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains generator to fool discriminator
        self.combined = Model(z, valid)
        # (!!!) Optimize w.r.t. MSE loss instead of crossentropy
        self.combined.compile(loss='mse', optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        # model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        # (!!!) No softmax
        model.add(Dense(1))
        # model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):
        
        if self.dataset_name == "mnist":
            (X_train, _), (_, _) = mnist.load_data()
            # Rescale -1 to 1
            X_train = (X_train.astype(np.float32) - 127.5) / 127.5
            X_train = np.expand_dims(X_train, axis=3)
        else:
            X_train = self.data_loader.load_data()
            # Rescale -1 to 1
            X_train = (X_train.astype(np.float32) - 127.5) / 127.5

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Sample noise as generator input
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


            # ---------------------
            #  Train Generator
            # ---------------------

            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print("epoch %d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch+1, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        os.makedirs('../result/%s/lsgan' % self.dataset_name, exist_ok=True)
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                if self.dataset_name == "mnist":
                    axs[i,j].imshow(gen_imgs[cnt,:,:,0], cmap='gray')
                else:
                    axs[i,j].imshow(gen_imgs[cnt,:,:,:])
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("../result/{}/lsgan/epoch{}.png".format(self.dataset_name, epoch),
                     transparent=True, dpi=300, bbox_inches="tight", pad_inches=0.0)
        plt.close()

In [8]:
lsgan = LSGAN(dataset_name="gratin")
# lsgan = LSGAN(dataset_name="mnist")
lsgan.train(epochs=30000, batch_size=32, sample_interval=200)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  app.launch_new_instance()


ストリーミング出力は最後の 5000 行に切り捨てられました。
epoch 25002 [D loss: 0.023152, acc.: 100.00%] [G loss: 1.121797]
epoch 25003 [D loss: 0.017064, acc.: 100.00%] [G loss: 1.187530]
epoch 25004 [D loss: 0.033397, acc.: 100.00%] [G loss: 0.950297]
epoch 25005 [D loss: 0.017036, acc.: 100.00%] [G loss: 1.057047]
epoch 25006 [D loss: 0.016786, acc.: 100.00%] [G loss: 1.087736]
epoch 25007 [D loss: 0.028907, acc.: 98.44%] [G loss: 1.089639]
epoch 25008 [D loss: 0.023751, acc.: 100.00%] [G loss: 1.005554]
epoch 25009 [D loss: 0.024408, acc.: 100.00%] [G loss: 1.161437]
epoch 25010 [D loss: 0.016607, acc.: 100.00%] [G loss: 1.103116]
epoch 25011 [D loss: 0.028080, acc.: 100.00%] [G loss: 1.072605]
epoch 25012 [D loss: 0.030344, acc.: 98.44%] [G loss: 1.059870]
epoch 25013 [D loss: 0.023471, acc.: 100.00%] [G loss: 1.109836]
epoch 25014 [D loss: 0.023949, acc.: 100.00%] [G loss: 1.179419]
epoch 25015 [D loss: 0.013237, acc.: 100.00%] [G loss: 1.025552]
epoch 25016 [D loss: 0.029002, acc.: 98.44%] [G loss: 1.1647